<a href="https://colab.research.google.com/github/clitwiniuk/Curso_Python/blob/master/EDA_Brocker_V2_2_(Carlos).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # Para la manipulación y análisis de datos
import numpy as np  # Para operaciones numéricas
import matplotlib.pyplot as plt  # Para la visualización de datos
import seaborn as sns  # Para la visualización de datos estadísticos
from google.colab import drive  # Para conectarse a Google Drive
import os  # Para interactuar con el sistema operativo
import re  # Para el procesamiento del texto

In [ ]:
import sqlite3

# Montamos Google Drive
drive.mount('/content/drive')

# Definimos la ruta al archivo de la base de datos
file_path = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/portapapeles_all_data.db'


  # Conectamos a la base de datos
conn = sqlite3.connect(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(file_path)

/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/portapapeles_all_data.db


In [ ]:
# Leer la tabla completa
data = pd.read_sql("SELECT * FROM clipboard_data", conn)

# Mostrar las primeras filas
print("Estructura original de los datos:")
print(data.head())

Estructura original de los datos:
   id                                            content            timestamp
0   1  "Ticker"\t"Sector"\t"Nombre"\t"Var"\t"%Var"\t"...  2024-12-11 21:33:54
1   2  "Ticker"\t"Sector"\t"Nombre"\t"Var"\t"%Var"\t"...  2024-12-11 21:34:12
2   3  "Ticker"\t"Sector"\t"Nombre"\t"Var"\t"%Var"\t"...  2024-12-11 21:34:31
3   4  "Ticker"\t"Sector"\t"Nombre"\t"Var"\t"%Var"\t"...  2024-12-11 21:34:49
4   5  "Ticker"\t"Sector"\t"Nombre"\t"Var"\t"%Var"\t"...  2024-12-11 21:35:08


In [ ]:
# Columnas finales requeridas
columnas_finales = ["Ticker", "Sector", "Nombre", "Var", "%Var", "Ratio vol", "Último",
                    "Volumen", "Capital", "Spread", "Inserción", "Anterior", "Vol V",
                    "Vol C", "Mínimo", "Máximo", "Compra", "Apertura", "Venta", "Timestamp"]

# Función para procesar 'content' y organizarlo correctamente
def procesar_content(row):
    """
    Procesa una fila de 'content' para extraer y alinear los datos con las columnas esperadas.
    """
    content = row['content']
    timestamp = row['timestamp']

    # Extraer todos los valores entre comillas dobles
    valores = re.findall(r'"(.*?)"', content)

    # Dividir los valores en bloques correspondientes a las columnas
    registros = []
    for i in range(0, len(valores), len(columnas_finales) - 1):  # Restamos 1 por 'Timestamp'
        registro = valores[i:i + len(columnas_finales) - 1]

        # Si faltan valores, completar con None (NaN)
        while len(registro) < len(columnas_finales) - 1:
            registro.append(None)

        # Agregar el timestamp al final del registro
        registro.append(timestamp)

        registros.append(registro)

    # Convertir los registros en un DataFrame
    temp_df = pd.DataFrame(registros, columns=columnas_finales)
    return temp_df

# Procesar todas las filas de la tabla
dfs_procesados = []

for _, row in data.iterrows():
    df_temp = procesar_content(row)
    dfs_procesados.append(df_temp)

# Concatenar los resultados
resultado_df = pd.concat(dfs_procesados, ignore_index=True)

# Mostrar el resultado
print("Resultado final alineado:")
print(resultado_df.head())

Resultado final alineado:
   Ticker           Sector            Nombre      Var     %Var  Ratio vol  \
0  Ticker           Sector            Nombre      Var     %Var  Ratio vol   
1    XPON      Industrials    EXPION360 INC.    -0,11   -5,47%   12,74489   
2    AMLI  Basic Materials  AMERICAN LITHIUM  -0,1888  -30,20%  12,268837   
3      TC      Industrials      TUANCHE LTD.  -0,0789   -8,58%  11,830085   
4    NXGL      Health Care        NEXGEL INC    +0,65  +18,47%  11,595072   

   Último  Volumen  Capital  Spread  Inserción  Anterior  Vol V  Vol C  \
0  Último  Volumen  Capital  Spread  Inserción  Anterior  Vol V  Vol C   
1    1,90  845.041   1.606k    0,04   22:28:27      2,01    300    100   
2  0,4364   6.343k   2.768k  0,0064   22:28:27    0,6252    100  2.000   
3  0,8411  632.577  532.060  0,0111   22:28:27    0,9200    100    200   
4    4,17  496.012   2.068k    0,17   22:28:27      3,52    500    900   

   Mínimo  Máximo  Compra  Apertura   Venta            Timestamp  

In [ ]:
# Eliminar filas donde 'Ticker' sea igual a 'Ticker' (nombres de columna repetidos)
resultado_df = resultado_df[resultado_df['Ticker'] != 'Ticker']

In [ ]:
# Nos aseguramos que cada celda del CSV se encuentre sobre el margen derecho, por cuestiones de estetica:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Creamos una primer columna de ID autonumerico para cada ticker
resultado_df['ID'] = range(1, len(resultado_df) + 1)

In [ ]:
# Exportar a CSV
output_path = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/All_data_reorganizado/all_data_reorganizado.csv'
resultado_df.to_csv(output_path, index=False)

print(f"Archivo CSV exportado correctamente a: {output_path}")

Archivo CSV exportado correctamente a: /content/drive/MyDrive/Proyecto_SmallCaps/BBDD/All_data_reorganizado/all_data_reorganizado.csv
